# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 56 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.09811


/tmp/ipykernel_2252/4030337529.py:34: LatexWarning: 2211.09811 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2211.09815


extracting tarball to tmp_2211.09815...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.10300


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.10297/KELT-9bWFC3_Jacobs_corr.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2211.10300...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.10300/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09840-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09840) | **Astrometric Accelerations as Dynamical Beacons: Discovery and  Characterization of HIP 21152 B, the First T-Dwarf Companion in the Hyades**  |
|| Kyle Franson, et al. -- incl., <mark>Matthias Samland</mark>, <mark>Zhoujian Zhang</mark>, <mark>Kevin Heng</mark> |
|*Appeared on*| *2022-11-21*|
|*Comments*| *40 pages, 19 figures, accepted to AJ*|
|**Abstract**| Benchmark brown dwarf companions with well-determined ages and model-independent masses are powerful tools to test substellar evolutionary models and probe the formation of giant planets and brown dwarfs. Here, we report the independent discovery of HIP~21152~B, the first imaged brown dwarf companion in the Hyades, and conduct a comprehensive orbital and atmospheric characterization of the system. HIP~21152 was targeted in an ongoing high-contrast imaging campaign of stars exhibiting proper motion changes between Hipparcos and Gaia, and was also recently identified by Bonavita et al. (2022) and Kuzuhara et al. (2022). Our Keck/NIRC2 and SCExAO/CHARIS imaging of HIP~21152 revealed a comoving companion at a separation of $0.37^{\prime\prime}$ (16 au). We perform a joint orbit fit of all available relative astrometry and radial velocities together with the Hipparcos-Gaia proper motions, yielding a dynamical mass of $24^{+6}_{-4}\,\mathrm{M_{Jup}}$, which is $1{-}2{\sigma}$ lower than evolutionary model predictions. Hybrid grids that include the evolution of cloud properties best reproduce the dynamical mass. We also identify a comoving wide-separation ($1837^{\prime\prime}$ or $7.9 \times 10^4 \, \mathrm{au}$) early-L dwarf with an inferred mass near the hydrogen-burning limit. Finally, we analyze the spectra and photometry of HIP~21152~B using the Saumon & Marley (2008) atmospheric models and a suite of retrievals. The best-fit grid-based models have $f_{\mathrm{sed}}=2$, indicating the presence of clouds, $T_{\mathrm{eff}}=1400 \, \mathrm{K}$, and $\log{g}=4.5 \, \mathrm{dex}$. These results are consistent with the object's spectral type of $\mathrm{T0\pm1}$. As the first benchmark brown dwarf companion in the Hyades, HIP~21152~B joins the small but growing number of substellar companions with well-determined ages and dynamical masses. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10215-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10215) | **CARMA-NRO Orion Survey: unbiased survey of dense cores and core mass  functions in Orion A**  |
|| Hideaki Takemura, et al. -- incl., <mark>Dariusz C. Lis Álvaro Sánchez-Monge</mark> |
|*Appeared on*| *2022-11-21*|
|*Comments*| *58 pages, 33 figures, 21 tables, accepted by ApJS*|
|**Abstract**| The mass distribution of dense cores is a potential key to understand the process of star formation. Applying dendrogram analysis to the CARMA-NRO Orion C$^{18}$O ($J$=1--0) data, we identify 2342 dense cores, about 22 \% of which have virial ratios smaller than 2, and can be classified as gravitationally bound cores. The derived core mass function (CMF) for bound starless cores which are not associate with protostars has a slope similar to Salpeter's initial mass function (IMF) for the mass range above 1 $M_\odot$, with a peak at $\sim$ 0.1 $M_\odot$. We divide the cloud into four parts based on the declination, OMC-1/2/3, OMC-4/5, L1641N/V380 Ori, and L1641C, and derive the CMFs in these regions. We find that starless cores with masses greater than 10 $M_\odot$ exist only in OMC-1/2/3, whereas the CMFs in OMC-4/5, L1641N, and L1641C are truncated at around 5--10 $M_\odot$. From the number ratio of bound starless cores and Class II objects in each subregion, the lifetime of bound starless cores is estimated to be 5--30 free-fall times, consistent with previous studies for other regions. In addition, we discuss core growth by mass accretion from the surrounding cloud material to explain the coincidence of peak masses between IMFs and CMFs. The mass accretion rate required for doubling the core mass within a core lifetime is larger than that of Bondi-Hoyle accretion by a factor of order 2. This implies that more dynamical accretion processes are required to grow cores. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10216-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10216) | **Spectroscopic and Imaging Observations of Spatially Extended Magnetic  Reconnection in the Splitting of a Solar Filament Structure**  |
|| Huidong Hu, et al. -- incl., <mark>Ying D. Liu</mark> |
|*Appeared on*| *2022-11-21*|
|*Comments*| *10 pages, 5 figures; an animation for Figure 1 available at this http URL ; published in ApJ Letters*|
|**Abstract**| On the Sun, Doppler shifts of bidirectional outflows from the magnetic-reconnection site have been found only in confined regions through spectroscopic observations. Without spatially resolved spectroscopic observations across an extended region, the distribution of reconnection and its outflows in the solar atmosphere cannot be made clear. Magnetic reconnection is thought to cause the splitting of filament structures, but unambiguous evidence has been elusive. Here we report spectroscopic and imaging analysis of a magnetic-reconnection event on the Sun, using high-resolution data from the Interface Region Imaging Spectrograph and the Solar Dynamics Observatory. Our findings reveal that the reconnection region extends to an unprecedented length of no less than 14,000 km. The reconnection splits a filament structure into two branches, and the upper branch erupts eventually. Doppler shifts indicate clear bidirectional outflows of ~100 km/s, which decelerate beyond the reconnection site. Differential-emission-measure analysis reveals that in the reconnection region the temperature reaches over 10 MK and the thermal energy is much larger than the kinetic energy. This Letter provides definite spectroscopic evidence for the splitting of a solar filament by magnetic reconnection in an extended region. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10236-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10236) | **The non-LTE formation of the Fe I 6173 A line in the solar atmosphere**  |
|| <mark>H. N. Smitha</mark>, M. van Noort, S. K. Solanki, J. S. Castellanos Durán |
|*Appeared on*| *2022-11-21*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| The current analysis is dedicated to a detailed investigation of the non-Local Thermodynamic Equilibrium (NLTE) effects influencing the formation of the Fe I 6173 A line, which is widely used by many instruments including the Helioseismic and Magnetic Imager (HMI) on-board the Solar Dynamics Observatory (SDO) and the Polarimetric and Helioseismic Imager on board the Solar Orbiter. We synthesize the Stokes profiles in a snapshot of a three dimensional magnetohydrodynamic simulation of the solar photosphere under both LTE and NLTE conditions. The simulation cube contains a sunspot and a plage region around it. The LTE and NLTE Stokes profiles formed in different features are compared and analysed. NLTE effects are evident in both intensity and polarization profiles. For the 6173 A line, UV overionization is the dominant NLTE mechanism, and scattering effects are much less important. In addition to Fe, an NLTE treatment of Si, Mg and Al is necessary to set the right photon density in the UV. This is found to further enhance the LTE departures compared to the case where Fe alone is treated in NLTE. These effects in the Stokes profiles survive even when the profiles are averaged spatially or sampled on a coarse wavelength grid such as that used by the SDO/HMI and other magnetographs. The deviations from the LTE profiles are stronger in the Fe I 6173 A compared to the 6301 A - 6302 A lines because in case of the latter, line scattering compensates the effect of UV overionization. Based on the nature of departures from LTE, treating the 6173 A line in LTE will likely result in an over-estimation of temperature and an under-estimation of the magnetic field strength. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10281-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10281) | **MeV-GeV Gamma-ray Emission from SNR G327.1-1.1 Discovered by the  Fermi-LAT**  |
|| Jordan Eagle, et al. -- incl., <mark>Joseph Gelfand</mark> |
|*Appeared on*| *2022-11-21*|
|*Comments*| *13 pages, 6 figures, 4 tables. Accepted for publication in ApJ on Oct. 31, 2022*|
|**Abstract**| We report the discovery of MeV-GeV gamma-ray emission by the Fermi-LAT positionally coincident with the TeV pulsar wind nebula (PWN) HESS~J1554-550 within the host supernova remnant (SNR) G327.1-1.1. The gamma-ray emission is point-like and faint but significant (> 4 sigma) in the 300MeV-2TeV energy range. We report here the Fermi-LAT analysis of the observed gamma-ray emission followed by a detailed multiwavelength investigation to understand the nature of the emission. The central pulsar powering the PWN within G327.1-1.1 has not been detected in any waveband; however, it is likely embedded within the X-ray nebula, which is displaced from the center of the radio nebula. The gamma-ray emission is faint and therefore a pulsation search to determine if the pulsar may be contributing is not feasible. Prior detailed multiwavelength reports revealed an SNR system that is old, tau ~ 18,000yrs, where the interaction of the reverse shock with the PWN is underway or has recently occurred. We find that the gamma-ray emission agrees remarkably well with a detailed broadband model constructed in a prior report based on independent hydrodynamical and semi-analytic simulations of an evolved PWN. We further investigate the physical implications of the model for the PWN evolutionary stage incorporating the new Fermi-LAT data and attempt to model the distinct particle components based on a spatial separation analysis of the displaced PWN counterparts. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10297-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10297) | **A strong H- opacity signal in the near-infrared emission spectrum of the  ultra-hot Jupiter KELT-9b**  |
|| Bob Jacobs, et al. -- incl., <mark>Michael R. Line</mark> |
|*Appeared on*| *2022-11-21*|
|*Comments*| *8 pages, 4 figures, Accepted for publication in A&A letters*|
|**Abstract**| We present the analysis of a spectroscopic secondary eclipse of the hottest transiting exoplanet detected to date, KELT-9b, obtained with the Wide Field Camera 3 aboard the Hubble Space Telescope. We complement these data with literature information on stellar pulsations and Spitzer/Infrared Array Camera and Transiting Exoplanet Survey Satellite eclipse depths of this target to obtain a broadband thermal emission spectrum. Our extracted spectrum exhibits a clear turnoff at 1.4$\mu$m. This points to H$^{-}$ bound-free opacities shaping the spectrum. To interpret the spectrum, we perform grid retrievals of self-consistent 1D equilibrium chemistry forward models, varying the composition and energy budget. The model with solar metallicity and C/O ratio provides a poor fit because the H$^{-}$ signal is stronger than expected, requiring an excess of electrons. This pushes our retrievals toward high atmospheric metallicities ($[M/H]=1.98^{+0.19}_{-0.21}$) and a C/O ratio that is subsolar by 2.4$\sigma$. We question the viability of forming such a high-metallicity planet, and therefore provide other scenarios to increase the electron density in this atmosphere. We also look at an alternative model in which we quench TiO and VO. This fit results in an atmosphere with a slightly subsolar metallicity and subsolar C/O ratio ($[M/H]=-0.22^{+0.17}_{-0.13}$, log(C/O)$=-0.34^{+0.19}_{-0.34}$). However, the required TiO abundances are disputed by recent high-resolution measurements of the same planet. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.10300-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.10300) | **The Microvariability and Wavelength Dependence of Polarization  Degree/Angle of BL Lacertae in the Outburst 2020 to 2021**  |
|| Ryo Imazawa, et al. -- incl., <mark>Mahito Sasada</mark> |
|*Appeared on*| *2022-11-21*|
|*Comments*| *19 pages, 8 figures*|
|**Abstract**| We have obtained simultaneous and continuous photo-polarization observations of the blazar BL Lacertae in optical and near-infrared (NIR) bands during a historical outburst from 2020 to 2021. In total, fourteen nights of observations were performed where ten observations show microvariability on timescales of a few minutes to several hours. This suggests a compact emission region, and the timescales are difficult to explain by a one-zone shock-in-jet model. Moreover, we found significant differences in the polarization degree (PD) and angle between optical and NIR bands. Nine nights showed a PD in the optical band that is greater than or equal to that in the NIR band, which can be explained by either a shock-in-jet model or the Turbulent Extreme Multi-Zone (TEMZ) model. On the other hand, five nights showed higher PD in a NIR band than an optical band, which cannot be explained by simple shock-in-jet models nor the simple TEMZ model. The observed timescales and wavelength dependency of the PD and polarization angle suggest the existence of complicated multiple emission regions such as an irregular TEMZ model. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09811-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09811) | **Visual Magnitude of the BlueWalker 3 Satellite**  |
|| <mark>Anthony Mallama</mark>, Richard E. Cole, Scott Harrington, Paul D. Maley |
|*Appeared on*| *2022-11-21*|
|*Comments*| **|
|**Abstract**| Observations have been carried out in order to assess the optical characteristics of the BlueWalker 3 spacecraft. The illumination phase function has been determined and evaluated. The average visual magnitude when seen overhead at the beginning or ending of astronomical twilight is found to be 1.5. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.09840.md
    + _build/html/tmp_2211.09840/figures/spectrum_both_matthias.png
    + _build/html/tmp_2211.09840/figures/ifs_reduction_comp.png
    + _build/html/tmp_2211.09840/figures/model_spec_comp_plot_matthias.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Based in part on data collected at Subaru Telescope, which is operated by the National Astronomical Observatory of Japan.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.09840-b31b1b.svg)](https://arxiv.org/abs/2211.09840)<mark>Appeared on: 2022-11-21</mark> - _40 pages, 19 figures, accepted to AJ_

</div>
<div id="authors">

Kyle Franson, et al. -- incl., <mark>Matthias Samland</mark>, <mark>Zhoujian Zhang</mark>, <mark>Kevin Heng</mark>

</div>
<div id="abstract">

**Abstract:** Benchmark brown dwarf companions with well-determined ages and model-independent masses are powerful tools to test substellar evolutionary models and probe the formation of giant planets and brown dwarfs. Here, we report the independent discovery of HIP 21152 B, the first imaged brown dwarf companion in the Hyades, and conduct a comprehensive orbital and atmospheric characterization of the system. HIP 21152 was targeted in an ongoing high-contrast imaging campaign of stars exhibiting proper motion changes between Hipparcos and Gaia, and was also recently identified by\citet{bonavitaResultsCopainsPilot_2022}and\citet{kuzuharaDirectImagingDiscovery_2022}. Our Keck/NIRC2 and SCExAO/CHARIS imaging of HIP 21152 revealed a comoving companion at a separation of$0$\farcs$37$(16 au). We perform a joint orbit fit of all available relative astrometry and radial velocities together with the Hipparcos-Gaia proper motions, yielding a dynamical mass of$24^{+6}_{-4} \mathrm{M_{Jup}}$, which is$1{-}2{\sigma}$lower than evolutionary model predictions. Hybrid grids that include the evolution of cloud properties best reproduce the dynamical mass. We also identify a comoving wide-separation ($1837$\arcsec$$or$\SI{7.9e4}{au}$) early-L dwarf with an inferred mass near the hydrogen-burning limit. Finally, we analyze the spectra and photometry of HIP 21152 B using the\citet{saumonEvolutionDwarfsColor_2008}atmospheric models and a suite of retrievals. The best-fit grid-based models have$f_{\mathrm{sed}}=2$, indicating the presence of clouds,$T_{\mathrm{eff}}=\SI{1400}{K}$, and$\log{g}=\SI{4.5}{dex}$. These results are consistent with the object's spectral type of$\mathrm{T0\pm1}$. As the first benchmark brown dwarf companion in the Hyades, HIP 21152 B joins the small but growing number of substellar companions with well-determined ages and dynamical masses.

</div>

<div id="div_fig1">

<img src="tmp_2211.09840/figures/spectrum_both_matthias.png" alt="Fig15" width="100%"/>

**Figure 15. -** Reduced chi-square ($\chi^2_\nu$; left) and goodness-of-fit ($G$; right) values for model spectra from \citet{saumonEvolutionDwarfsColor_2008} compared with HIP 21152 B's spectra and photometry. Lower values of $\chi^2_\nu$ or $G$ signify a better fit to the data. Colors indicate different values of $f_{\mathrm{sed}}$, while symbols correspond to different values of $\log g$. Reduction 2 is used for the SPHERE/IFS spectrum. The two metrics yield the same best-fitting spectrum, with $T_{\mathrm{eff}} = \SI{1400}{K}$, $\log g = 4.5$, and $f_{\mathrm{sed}} = 2$.
    \label{fig:model_spec_comp} (*fig:model_spec_comp*)

</div>
<div id="div_fig2">

<img src="tmp_2211.09840/figures/ifs_reduction_comp.png" alt="Fig2" width="100%"/>

**Figure 2. -** Comparison between two independent reductions of the SPHERE/IFS data. Reduction 1 uses the DRH pipeline complemented with additional steps described in \citet{langloisSphereInfraredSurvey_2021} and the SpeCal software to extract spectral cubes from the raw data, subtract the host star PSF, and obtain the companion's spectrum. Reduction 2 uses a modified version of the CHARIS pipeline and the TRAP post-processing pipeline to measure the companion's spectrum from the raw data. These two independent reductions produce similar results for $\lambda \gtrsim \SI{1.15}{\mu m}$. Reduction 2 tends to yield lower values for the companion flux blueward of $\SI{1.15}{\mu m}$ than Reduction 1. Due to this discrepancy, we exclude points with $\lambda < \SI{1.15}{\mu m}$ from this analysis. \label{fig:spec_comp} (*fig:spec_comp*)

</div>
<div id="div_fig3">

<img src="tmp_2211.09840/figures/model_spec_comp_plot_matthias.png" alt="Fig5" width="100%"/>

**Figure 5. -** Spectra and photometry of HIP 21152 B compared with the best-fitting \citet{saumonEvolutionDwarfsColor_2008} model spectrum (black). The model represents $T_{\mathrm{eff}} = \SI{1400}{K}$, $\log{g} = \SI{4.5}{dex}$, and $f_{\mathrm{sed}} = 2$, and has been smoothed to a resolving power of $R=25$. Reduction 2 is used for the SPHERE/IFS spectrum. The atmospheric model reproduces the companion's spectra well, although we find a slight excess in $L^\prime$ that might hint at disequilibrium chemistry from 3--\SI{4}{\mu m}.
    \label{fig:spectrum_model_plot} (*fig:spectrum_model_plot*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

95  publications in the last 7 days.
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers